In [10]:
import firebase_admin
from firebase_admin import credentials, firestore
from google.cloud.firestore_v1.base_query import FieldFilter, Or
import os

In [4]:

cred = credentials.Certificate(f"./static/firebase_cred.json")
firebase_admin.initialize_app(cred)
db = firestore.client()
print(db)

In [5]:
class City:
    def __init__(self, name, state, country, capital=False, population=0, regions=[]):
        self.name = name
        self.state = state
        self.country = country
        self.capital = capital
        self.population = population
        self.regions = regions

    @staticmethod
    def from_dict(source: dict):
      if type(source) != dict:
        return None

      if "name" not in source \
      or "state" not in source \
      or "country" not in source:
        return None
        
      city = City(source["name"], source["state"], source["country"])

      if "capital" in source:
        city.capital = source["capital"]

      if "population" in source:
        city.population = source["population"]

      if "regions" in source:
        city.regions = source["regions"]

      return city

    def to_dict(self) -> dict:
      d = {}
      d["name"] = self.name
      d["state"] = self.state
      d["country"] = self.country
      d["capital"] = self.capital
      d["population"] = self.population
      d["regions"] = self.regions
      return d


    def __repr__(self):
        return f"City(\
                name={self.name}, \
                country={self.country}, \
                population={self.population}, \
                capital={self.capital}, \
                regions={self.regions}\
            )"


In [ ]:
cities_ref = db.collection("cities")
cities_ref.document("BJ").set(
    City("Beijing", None, "China", True, 21500000, ["hebei"]).to_dict()
)
cities_ref.document("SF").set(
    City(
        "San Francisco", "CA", "USA", False, 860000, ["west_coast", "norcal"]
    ).to_dict()
)
cities_ref.document("LA").set(
    City(
        "Los Angeles", "CA", "USA", False, 3900000, ["west_coast", "socal"]
    ).to_dict()
)
cities_ref.document("DC").set(
    City("Washington D.C.", None, "USA", True, 680000, ["east_coast"]).to_dict()
)
cities_ref.document("TOK").set(
    City("Tokyo", None, "Japan", True, 9000000, ["kanto", "honshu"]).to_dict()
)

In [11]:
cities_ref = db.collection("cities")

docs = cities_ref.where(
    filter=FieldFilter("regions", "array_contains", "west_coast")
).get()

for doc in docs:
  print(f"{doc.id} => {doc.to_dict()}")


LA => {'capital': False, 'name': 'Los Angeles', 'state': 'CA', 'regions': ['west_coast', 'socal'], 'population': 3900000, 'country': 'USA'}
SF => {'capital': False, 'name': 'San Francisco', 'state': 'CA', 'regions': ['west_coast', 'norcal'], 'population': 860000, 'country': 'USA'}


In [15]:

cities_ref = db.collection("cities")
docs = cities_ref.where(filter=FieldFilter("country", "in", ["Japan", "China"])).get()

for doc in docs:
  print(f"{doc.id} => {doc.to_dict()}")


BJ => {'capital': True, 'name': 'Beijing', 'state': None, 'regions': ['hebei'], 'population': 21500000, 'country': 'China'}
TOK => {'capital': True, 'name': 'Tokyo', 'state': None, 'regions': ['kanto', 'honshu'], 'population': 9000000, 'country': 'Japan'}


In [17]:
docs = cities_ref.where(
    filter=FieldFilter(
        "regions", "array_contains_any", ["west_coast", "east_coast"]
    )
).get()

for doc in docs:
  print(f"{doc.id} => {doc.to_dict()}")

DC => {'capital': True, 'name': 'Washington D.C.', 'state': None, 'regions': ['east_coast'], 'population': 680000, 'country': 'USA'}
LA => {'capital': False, 'name': 'Los Angeles', 'state': 'CA', 'regions': ['west_coast', 'socal'], 'population': 3900000, 'country': 'USA'}
SF => {'capital': False, 'name': 'San Francisco', 'state': 'CA', 'regions': ['west_coast', 'norcal'], 'population': 860000, 'country': 'USA'}


In [21]:
large_us_cities_query = cities_ref.where(
    filter=FieldFilter("state", "==", "CA")
).stream()

for doc in large_us_cities_query:
  if doc.to_dict()["population"] > 1000000:
    print(f"{doc.id} => {doc.to_dict()}")

LA => {'capital': False, 'name': 'Los Angeles', 'state': 'CA', 'regions': ['west_coast', 'socal'], 'population': 3900000, 'country': 'USA'}


In [23]:
col_ref = db.collection("cities")

filter1 = FieldFilter("country", "==", "China")
filter2 = FieldFilter("population", "==", "Japan")

or_filter = Or(filters=[filter1, filter2])

docs = cities_ref.where(
    filter=or_filter
).get()

for doc in docs:
  print(f"{doc.id} => {doc.to_dict()}")

FailedPrecondition: 400 The query requires an index. You can create it here: https://console.firebase.google.com/v1/r/project/flaskweek-pse-2023-1112/firestore/indexes?create_composite=ClZwcm9qZWN0cy9mbGFza3dlZWstcHNlLTIwMjMtMTExMi9kYXRhYmFzZXMvKGRlZmF1bHQpL2NvbGxlY3Rpb25Hcm91cHMvY2l0aWVzL2luZGV4ZXMvXxABGgsKB2NvdW50cnkQARoOCgpwb3B1bGF0aW9uEAEaDAoIX19uYW1lX18QAQ

In [26]:
cities_ref = db.collection("cities")
query = cities_ref.where(filter=FieldFilter("population", ">", 2500000)).order_by(
    "population"
)
results = query.stream()

for doc in results:
   print(f"{doc.id} => {doc.to_dict()}")

LA => {'capital': False, 'name': 'Los Angeles', 'state': 'CA', 'regions': ['west_coast', 'socal'], 'population': 3900000, 'country': 'USA'}
TOK => {'capital': True, 'name': 'Tokyo', 'state': None, 'regions': ['kanto', 'honshu'], 'population': 9000000, 'country': 'Japan'}
BJ => {'capital': True, 'name': 'Beijing', 'state': None, 'regions': ['hebei'], 'population': 21500000, 'country': 'China'}
